In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "/content/drive/MyDrive/Colab Notebooks/platform_subscribe/"

train = pd.read_csv(base_path + "train.csv")
test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "sample_submission.csv")

In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.6 MB/s eta 0:00:00


In [4]:
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

evulation_metric = f1_score

In [5]:
X=train.drop(columns=['user_id','target'])
y=train.target

In [6]:
from sklearn.model_selection import train_test_split
# 트테트테
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)

# 6:2:2 = train: validation:test
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(6000, 13) (2000, 13) (2000, 13) (6000,) (2000,) (2000,)


### feature importance catboostclassifier 이용해 확인

In [7]:
clf_cb = CatBoostClassifier()
clf_cb.fit(X_train, y_train, cat_features=['preferred_difficulty_level', 'subscription_type'])

importances = clf_cb.get_feature_importance(type='PredictionValuesChange')
feature_importances = pd.Series(importances, index=X.columns).sort_values(ascending=False)

Learning rate set to 0.022141
0:	learn: 0.6910398	total: 69.6ms	remaining: 1m 9s
1:	learn: 0.6884479	total: 85.5ms	remaining: 42.7s
2:	learn: 0.6858057	total: 101ms	remaining: 33.7s
3:	learn: 0.6833770	total: 122ms	remaining: 30.4s
4:	learn: 0.6810093	total: 141ms	remaining: 28.1s
5:	learn: 0.6791555	total: 159ms	remaining: 26.4s
6:	learn: 0.6775521	total: 175ms	remaining: 24.8s
7:	learn: 0.6754125	total: 199ms	remaining: 24.7s
8:	learn: 0.6734520	total: 216ms	remaining: 23.8s
9:	learn: 0.6716197	total: 232ms	remaining: 23s
10:	learn: 0.6699652	total: 242ms	remaining: 21.7s
11:	learn: 0.6689154	total: 259ms	remaining: 21.3s
12:	learn: 0.6682318	total: 267ms	remaining: 20.2s
13:	learn: 0.6671348	total: 278ms	remaining: 19.6s
14:	learn: 0.6656228	total: 286ms	remaining: 18.8s
15:	learn: 0.6641094	total: 295ms	remaining: 18.2s
16:	learn: 0.6631681	total: 302ms	remaining: 17.5s
17:	learn: 0.6624932	total: 313ms	remaining: 17.1s
18:	learn: 0.6614418	total: 320ms	remaining: 16.5s
19:	learn: 

In [8]:
feature_importances

average_time_per_learning_session    24.193947
recent_learning_achievement          11.074801
average_login_time                   10.270656
subscription_duration                 8.692782
recent_login_time                     7.459148
monthly_active_learning_days          7.010165
total_completed_courses               6.859084
abandoned_learning_sessions           5.411463
payment_pattern                       4.956338
customer_inquiry_history              4.349659
preferred_difficulty_level            4.286928
community_engagement_level            3.142085
subscription_type                     2.292945
dtype: float64

In [9]:
pred_val_cb=clf_cb.predict(X_val)
print('Catboost : ' , evulation_metric(y_val, pred_val_cb))
print(pred_val_cb)

Catboost :  0.7468639433901576
[1 1 1 ... 1 0 1]


In [10]:
pred_test_cb=clf_cb.predict(X_test)
print('Catboost : ' , evulation_metric(y_val, pred_test_cb))
print(pred_test_cb)

Catboost :  0.7464424320827943
[1 1 1 ... 1 1 1]


### 확인한 feature importance 활용해 feature 개수 별로 성능 좋은 거 확인해보기

In [11]:
feature_importances_append=feature_importances.index
feature_importances_append[0]
feature_columns =[]
evulation_metric_result = {}
for i in range(0,10):
  feature_columns.append(feature_importances_append[i])
  X=train[feature_columns]
  y=train.target
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
  X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)
  clf_cb = CatBoostClassifier()
  clf_cb.fit(X_train, y_train)
  pred_val_cb=clf_cb.predict(X_val)
  evulation_metric_cb=evulation_metric(y_val, pred_val_cb)
  print(evulation_metric_cb)
  evulation_metric_result[tuple(feature_columns)] = evulation_metric_cb


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
9:	learn: 0.6727083	total: 33.5ms	remaining: 3.31s
10:	learn: 0.6710977	total: 42.2ms	remaining: 3.79s
11:	learn: 0.6700750	total: 46.8ms	remaining: 3.85s
12:	learn: 0.6687915	total: 49.7ms	remaining: 3.78s
13:	learn: 0.6676132	total: 53ms	remaining: 3.73s
14:	learn: 0.6661312	total: 55.9ms	remaining: 3.67s
15:	learn: 0.6649522	total: 59.1ms	remaining: 3.63s
16:	learn: 0.6639522	total: 62.2ms	remaining: 3.6s
17:	learn: 0.6631267	total: 65.3ms	remaining: 3.56s
18:	learn: 0.6624903	total: 68.3ms	remaining: 3.52s
19:	learn: 0.6614839	total: 71.4ms	remaining: 3.5s
20:	learn: 0.6604635	total: 74.4ms	remaining: 3.47s
21:	learn: 0.6598095	total: 77.4ms	remaining: 3.44s
22:	learn: 0.6586837	total: 80.4ms	remaining: 3.41s
23:	learn: 0.6581743	total: 83.7ms	remaining: 3.4s
24:	learn: 0.6576840	total: 86.6ms	remaining: 3.38s
25:	learn: 0.6571573	total: 89.8ms	remaining: 3.36s
26:	learn: 0.6560432	total: 92.9ms	remaining: 3.35s
27:	learn: 0.6551508	total: 95.9ms

In [12]:
sorted_evulation_metric_result = sorted(evulation_metric_result.items(), key=lambda x:x[1], reverse=True)
sorted_evulation_metric_result

[(('average_time_per_learning_session',), 0.7599243856332704),
 (('average_time_per_learning_session', 'recent_learning_achievement'),
  0.7491188721563603),
 (('average_time_per_learning_session',
   'recent_learning_achievement',
   'average_login_time',
   'subscription_duration',
   'recent_login_time',
   'monthly_active_learning_days',
   'total_completed_courses',
   'abandoned_learning_sessions',
   'payment_pattern',
   'customer_inquiry_history'),
  0.7396275726886639),
 (('average_time_per_learning_session',
   'recent_learning_achievement',
   'average_login_time',
   'subscription_duration',
   'recent_login_time',
   'monthly_active_learning_days',
   'total_completed_courses',
   'abandoned_learning_sessions',
   'payment_pattern'),
  0.7390589009542612),
 (('average_time_per_learning_session',
   'recent_learning_achievement',
   'average_login_time',
   'subscription_duration',
   'recent_login_time',
   'monthly_active_learning_days',
   'total_completed_courses'),
  

In [13]:
X=train[['average_time_per_learning_session']]
y=train.target
X

,average_time_per_learning_session
0,8.427187
1,72.646087
2,21.774492
3,42.659066
4,30.744287
...,...
9995,84.053558
9996,45.464833
9997,127.302411
9998,5.297234


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)
clf_cb = CatBoostClassifier()
clf_cb.fit(X_train, y_train)
pred_train_cb=clf_cb.predict(X_train)
pred_val_cb=clf_cb.predict(X_val)

evulation_metric_train_cb=evulation_metric(y_train, pred_train_cb)
evulation_metric_val_cb=evulation_metric(y_val, pred_val_cb)
print(evulation_metric_train_cb)
print(evulation_metric_val_cb)

Learning rate set to 0.022141
0:	learn: 0.6901212	total: 2.51ms	remaining: 2.51s
1:	learn: 0.6869915	total: 5.28ms	remaining: 2.63s
2:	learn: 0.6842181	total: 7.51ms	remaining: 2.5s
3:	learn: 0.6822181	total: 10.3ms	remaining: 2.56s
4:	learn: 0.6797222	total: 12.5ms	remaining: 2.49s
5:	learn: 0.6774541	total: 15ms	remaining: 2.48s
6:	learn: 0.6762498	total: 17.5ms	remaining: 2.48s
7:	learn: 0.6750232	total: 20.4ms	remaining: 2.53s
8:	learn: 0.6739489	total: 22.9ms	remaining: 2.52s
9:	learn: 0.6723258	total: 25.4ms	remaining: 2.51s
10:	learn: 0.6711206	total: 27.6ms	remaining: 2.48s
11:	learn: 0.6691038	total: 30.2ms	remaining: 2.49s
12:	learn: 0.6677544	total: 32.7ms	remaining: 2.48s
13:	learn: 0.6668817	total: 35.1ms	remaining: 2.47s
14:	learn: 0.6657826	total: 37.5ms	remaining: 2.46s
15:	learn: 0.6648755	total: 40ms	remaining: 2.46s
16:	learn: 0.6637659	total: 42.5ms	remaining: 2.46s
17:	learn: 0.6627573	total: 45.1ms	remaining: 2.46s
18:	learn: 0.6615588	total: 48ms	remaining: 2.48s

### test 확인

In [22]:
pred_test_cb=clf_cb.predict(X_test)

evulation_metric_test_cb=evulation_metric(y_test, pred_test_cb)
print(evulation_metric_test_cb)

0.7585772741580107


### submission

In [16]:
X_submission=test[['average_time_per_learning_session']]
print(X_submission)
preds=clf_cb.predict(X_submission)
preds

      average_time_per_learning_session
0                              7.047243
1                             24.583327
2                              1.707616
3                             21.391389
4                             18.126987
...                                 ...
9995                          90.251023
9996                          10.205804
9997                          20.346781
9998                         156.521711
9999                          53.633453

[10000 rows x 1 columns]


array([1, 1, 1, ..., 1, 1, 1])

In [17]:
submission['target']=preds
submission

,user_id,target
0,0001d6e9,1
1,0002c77d,1
2,0002df5b,1
3,000b6068,1
4,00184a0c,1
...,...,...
9995,ffe2eba5,1
9996,ffe710f1,1
9997,ffeccdef,1
9998,fff3fcea,1


In [23]:
unique, counts = np.unique(preds, return_counts = True)
cnt_dict = dict(zip(unique, counts))
cnt_dict

{0: 294, 1: 9706}

In [24]:
submission.to_csv(base_path+"submission_1209_3.csv", index=False)